# Step 1: Mounting Google Drive and Importing Libraries


In [33]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/grpo-verified-reasoner
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/grpo-verified-reasoner
data			      LICENSE	 outputs    unsloth_compiled_cache
grpo_trainer_lora_model       models	 README.md  _unsloth_sentencepiece_temp
huggingface_tokenizers_cache  notebooks  src	    wandb


In [ ]:
# Install UV (Faster pip)
!pip install --upgrade -qqq uv

In [4]:
import os
import subprocess

In [35]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:False"

In [36]:
# os.environ["UNSLOTH_VLLM_STANDBY"] = "1"

In [6]:
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install -q unsloth vllm human-eval tqdm
else:
    # Version matching for Colab GPUs
    try:
        import numpy, PIL
        get_numpy = f"numpy=={numpy.__version__}"
        get_pil   = f"pillow=={PIL.__version__}"
    except Exception:
        get_numpy, get_pil = "numpy", "pillow"

    try:
        is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except Exception:
        is_t4 = False

    # A100/H100: vllm 0.10.2, T4: vllm 0.9.2 + pinned triton
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")

    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers tqdm human-eval
    !uv pip install -qqq {get_triton}

In [13]:
import gc
import json
import re
import ast
import torch
import random
import textwrap
import numpy as np
from tqdm import tqdm

from unsloth import FastLanguageModel
from vllm import SamplingParams
from pathlib import Path

from human_eval.data import read_problems, write_jsonl
from human_eval.evaluation import evaluate_functional_correctness

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-02 11:29:00 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [14]:
SEED = 3407
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-80GB (UUID: GPU-bb257f27-7b11-6040-93de-72c8cff7bb91)


# Step 2: Verifying GPU and Environment

In [15]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Torch version: 2.8.0+cu128
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB


# Step 3: Setting Up the Main Variables

In [16]:
# HumanEval evaluation settings
N_SAMPLES_PER_PROBLEM = 1          # pass@1 by default; raise to 5 or 10 if you later want pass@k
MAX_NEW_TOKENS_NON_COT = 512       # Non-CoT completions are usually short (function body)
MAX_NEW_TOKENS_COT = 2048           # CoT can be longer due to tags + full function

TEMP_NON_COT = 0.0                # low sampling noise; stable for benchmarking
TEMP_COT = 0.0                     # encourages exploration under schema (optional)

TOP_P = 0.95
MIN_P = 0.10

# Stop guards (prevent rambling without clipping typical solutions)
STOP_STRINGS = ["\nclass ", "\ndef ", "\nif __name__"]

In [17]:
# Output paths (keep all artifacts under one folder)
EVAL_DIR = "data/evaluation"
os.makedirs(EVAL_DIR, exist_ok=True)

In [30]:
# Model paths / identifiers
BASE_MODEL_PATH = "unsloth/Qwen3-4B-Base"
SFT_MODEL_PATH  = "models/qwen3-4b-sft"
GRPO_MODEL_PATH = "models/qwen3-4b-grpo-final"

In [19]:
COT_SYSTEM_PROMPT_HUMANEVAL = """You are a code-generation engine.
You must output your response in the following exact format:
<START_WORKING_OUT>
Concise reasoning steps required to solve the problem.
</END_WORKING_OUT>
<SOLUTION>
Valid Python code only.
</SOLUTION>
Do not output anything outside these tags."""

# Step 4: Loading HumanEval Problems

In [20]:
problems = read_problems()  # dict: {task_id: {"prompt":..., "test":..., "entry_point":...}}
task_ids = list(problems.keys())

print(f"Loaded HumanEval problems: {len(task_ids)}")
print("Example task_id:", task_ids[0])
print("\n--- Prompt Preview ---")
print(problems[task_ids[0]]["prompt"][:500])

Loaded HumanEval problems: 164
Example task_id: HumanEval/0

--- Prompt Preview ---
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



# Step 5: Prompt Builders

In [21]:
def build_non_cot_prompt(problem: dict) -> str:
    """
    Non-CoT: HumanEval-style continuation.
    We provide ONLY the HumanEval prompt.
    The harness will prepend this prompt again during execution.
    """
    return problem["prompt"]

In [22]:
def build_cot_prompt(problem: dict, tokenizer) -> str:
    """
    CoT: Uses the same chat template distribution as SFT/GRPO training.
    Returns a fully formatted ChatML prompt string.
    """
    messages = [
        {"role": "system", "content": COT_SYSTEM_PROMPT_HUMANEVAL},
        {"role": "user", "content": problem["prompt"]},
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

# Step 6: Output Post-processing Logic

In [23]:
# Stop strings (for safety against rambling)
# These are conservative: they stop the model from starting a NEW definition / class / main block.
STOP_STRINGS = ["\nclass ", "\ndef ", "\nif __name__"]

SOLUTION_RE = re.compile(r"<SOLUTION>(.*?)</SOLUTION>", re.DOTALL | re.IGNORECASE)

In [24]:

def extract_cot_solution(completion_only_text: str) -> str:
    """
    Extract code from <SOLUTION>...</SOLUTION>.
    Returns "" on failure (schema violation → harness will fail, which is correct behavior).
    """
    m = SOLUTION_RE.search(completion_only_text)
    if not m:
        return ""
    return m.group(1).strip()

In [25]:
def truncate_on_stop_strings(text: str, stop_strings: list[str]) -> str:
    """
    Stops the completion if it begins a new unrelated block (def/class/main).
    This reduces harness crashes from rambling continuations.
    """
    cut = len(text)
    for s in stop_strings:
        idx = text.find(s)
        if idx != -1:
            cut = min(cut, idx)
    return text[:cut].rstrip()

In [26]:
def cleaner(code: str, entry_point: str, is_cot: bool) -> str:
    # A. EXTRACT CoT
    if is_cot:
        if "<SOLUTION>" in code:
            code = code.split("<SOLUTION>")[-1]
            if "</SOLUTION>" in code:
                code = code.split("</SOLUTION>")[0]
        else:
            return ""

    # B. REMOVE MARKDOWN
    code = code.replace("```python", "").replace("```", "")

    # C. FILTER LINES
    lines = code.split('\n')
    filtered_lines = []

    def_pattern = re.compile(rf"^\s*def\s+{re.escape(entry_point)}(\s*\(|\s*:)")

    for line in lines:
        # Remove redundant definition
        if def_pattern.match(line):
            continue
        # Remove "from typing" (Always redundant in HumanEval, creates noise)
        if line.strip().startswith("from typing"):
            continue
        filtered_lines.append(line)

    code = "\n".join(filtered_lines)

    # D. REMOVE DOCSTRINGS
    code = re.sub(r'(\s*("""|\'\'\')[\s\S]*?\2)', '', code, count=1)
    code = code.lstrip('\n\r')

    # E. SMART RELATIVE NORMALIZATION (The Fix)
    if code.strip():
        lines = code.split('\n')

        # 1. Find the "Reference Line" to measure indentation.
        # We skip lines starting with 'import' so we measure the ACTUAL code body.
        reference_line = None
        for l in lines:
            stripped = l.strip()
            if stripped and not stripped.startswith(("import ", "from ")):
                reference_line = l
                break

        # Fallback: If no code found (only imports?), use the first line.
        if reference_line is None:
            reference_line = next((l for l in lines if l.strip()), None)

        if reference_line:
            # Measure indentation of the BODY
            body_indent = len(reference_line) - len(reference_line.lstrip())

            normalized_lines = []
            for line in lines:
                # If the line is empty, just strip it
                if not line.strip():
                    normalized_lines.append("")
                    continue

                # Calculate current indent
                current_indent = len(line) - len(line.lstrip())

                # Subtract the body_indent.
                # If an import is at 0 and body is at 4, this becomes -4.
                # We max(0, ...) to ensure we don't crash, effectively pulling the import in.
                new_indent = max(0, current_indent - body_indent)

                # Reconstruct the line
                normalized_lines.append(" " * new_indent + line.lstrip())

            code = "\n".join(normalized_lines)

        # Finally, indent everything by 4 for the harness
        code = textwrap.indent(code, '    ')

    return code

# Step 7: Defining Evaluation Loop

In [27]:
def evaluate_model(
    model_path: str,
    problems: dict,
    task_ids: list[str],
    *,
    use_cot: bool,
    output_jsonl: str,
    max_new_tokens: int,
    temperature: float,
    top_p: float = 0.95,
    min_p: float = 0.10,
    load_in_4bit: bool = True,
    gpu_memory_utilization: float = 0.7,
):
    print(f"\n Loading: {model_path} | Mode: {'CoT' if use_cot else 'Non-CoT'}")

    gc.collect()
    torch.cuda.empty_cache()

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_path,
        max_seq_length=4096,
        load_in_4bit=load_in_4bit,
        fast_inference=True,
        gpu_memory_utilization=gpu_memory_utilization,
    )
    FastLanguageModel.for_inference(model)

    prompts = []
    print(f" Preparing {len(task_ids)} prompts...")
    for task_id in task_ids:
        problem = problems[task_id]
        if use_cot:
            prompt_text = build_cot_prompt(problem, tokenizer)
        else:
            prompt_text = build_non_cot_prompt(problem)
        prompts.append(prompt_text)

    # Auto-Switching Stop Logic
    if use_cot:
        stop_tokens = ["</SOLUTION>"]
    else:
        stop_tokens = ["\nclass", "\nif __name__", "\nprint", "\ndef "]

    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        min_p=min_p,
        max_tokens=max_new_tokens,
        stop=stop_tokens,
        # repetition_penalty REMOVED per orders
    )

    print(f" Running vLLM Batch Generation...")
    outputs = model.fast_generate(prompts, sampling_params=sampling_params)

    samples = []
    for i, task_id in enumerate(task_ids):
        problem = problems[task_id]
        completion_only = outputs[i].outputs[0].text

        completion = cleaner(completion_only, problem["entry_point"], use_cot)

        samples.append({
            "task_id": task_id,
            "prompt": problem["prompt"],
            "completion": completion
        })

    write_jsonl(output_jsonl, samples)
    print(f" Saved {len(samples)} samples to: {output_jsonl}")

    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()

# Step 8: Generating JSONL Files for Evaluation

In [28]:
GEN_DIR = Path(EVAL_DIR) / "generations"
GEN_DIR.mkdir(parents=True, exist_ok=True)

In [36]:
# -------------------------
# BASE MODEL (Non-CoT)
# -------------------------
evaluate_model(
    model_path=BASE_MODEL_PATH,
    problems=problems,
    task_ids=task_ids,
    use_cot=False,
    output_jsonl=str(GEN_DIR / "base_non_cot.jsonl"),
    max_new_tokens=MAX_NEW_TOKENS_NON_COT,
    temperature=TEMP_NON_COT,
)


 Loading: unsloth/Qwen3-4B-Base | Mode: Non-CoT
INFO 01-02 11:37:40 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 01-02 11:37:40 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3. vLLM: 0.10.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-base-unsloth-bnb-4bit with actual GPU utilization = 19.78%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 79.32 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 64.
Unsloth: vLLM's KV Cache can use up to 12.83 GB.

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-02 11:38:02 [gpu_model_runner.py:2392] Model loading took 3.3523 GiB and 2.875338 seconds
INFO 01-02 11:38:17 [backends.py:539] Using cache directory: /root/.cache/vllm/torch_compile_cache/0550210604/rank_0_0/backbone for vLLM's torch.compile
INFO 01-02 11:38:17 [backends.py:550] Dynamo bytecode transform time: 14.14 s
INFO 01-02 11:38:23 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 5.087 s
INFO 01-02 11:38:26 [monitor.py:34] torch.compile takes 14.14 s in total
INFO 01-02 11:38:28 [gpu_worker.py:298] Available KV cache memory: 11.71 GiB
INFO 01-02 11:38:29 [kv_cache_utils.py:864] GPU KV cache size: 85,296 tokens
INFO 01-02 11:38:29 [kv_cache_utils.py:868] Maximum concurrency for 4,096 tokens per request: 20.82x
INFO 01-02 11:38:29 [vllm_utils.py:707] Unsloth: Running patched vLLM v1 `capture_model`.
INFO 01-02 11:38:29 [vllm_utils.py:707] Unsloth: Running patched vLLM v1 `capture_model`.


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 19/19 [00:06<00:00,  3.07it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 11/11 [00:03<00:00,  2.95it/s]

...

Some weights of Qwen3ForCausalLM were not initialized from the model checkpoint at unsloth/qwen3-4b-base-unsloth-bnb-4bit and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Performing substitution for additional_keys=set()
Unsloth: Just some info: will skip parsing ['layer_norm2', 'cross_attn_input_layernorm', 'q_norm', 'post_feedforward_layernorm', 'pre_feedforward_layernorm', 'norm1', 'ffn_norm', 'k_norm', 'layer_norm1', 'norm', 'cross_attn_post_attention_layernorm', 'input_layernorm', 'post_attention_layernorm', 'norm2', 'attention_norm', 'post_layernorm']
 Preparing 164 prompts...
 Running vLLM Batch Generation...


Adding requests:   0%|          | 0/164 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/164 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Saved 164 samples to: data/evaluation/generations/base_non_cot.jsonl


In [35]:
# -------------------------
# SFT MODEL (Non-CoT)
# -------------------------
evaluate_model(
    model_path=SFT_MODEL_PATH,
    problems=problems,
    task_ids=task_ids,
    use_cot=False,
    output_jsonl=str(GEN_DIR / "sft_non_cot.jsonl"),
    max_new_tokens=MAX_NEW_TOKENS_NON_COT,
    temperature=TEMP_NON_COT,
)


 Loading: models/qwen3-4b-sft | Mode: Non-CoT
INFO 01-02 11:33:21 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 01-02 11:33:21 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3. vLLM: 0.10.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-base-unsloth-bnb-4bit with actual GPU utilization = 69.6%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 79.32 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 52.34 GB. A

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 01-02 11:33:50 [__init__.py:1815] Using max model len 4096
WARNING 01-02 11:33:50 [_ipex_ops.py:16] Import error msg: No module named 'intel_extension_for_pytorch'
INFO 01-02 11:33:53 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 01-02 11:33:53 [lora.py:92] `lora_extra_vocab_size` is deprecated and will be removed in v0.12.0. Additional vocabulary support for LoRA adapters is being phased out.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.0.mlp', 'model.layers.4.mlp', 'model.layers.3.self_attn', 'model.layers.0.self_attn', 'model.layers.6.mlp', 'model.layers.1.self_attn', 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

Unsloth 2025.12.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


 Preparing 164 prompts...
 Running vLLM Batch Generation...


Adding requests:   0%|          | 0/164 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/164 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Saved 164 samples to: data/evaluation/generations/sft_non_cot.jsonl


In [31]:
# -------------------------
# SFT MODEL (CoT)
# -------------------------
evaluate_model(
    model_path=SFT_MODEL_PATH,
    problems=problems,
    task_ids=task_ids,
    use_cot=True,
    output_jsonl=str(GEN_DIR / "sft_cot.jsonl"),
    max_new_tokens=MAX_NEW_TOKENS_COT,
    temperature=TEMP_COT,
)


 Loading: models/qwen3-4b-sft | Mode: CoT
INFO 01-02 11:15:28 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 01-02 11:15:28 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3. vLLM: 0.10.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-base-unsloth-bnb-4bit with actual GPU utilization = 6.49%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 79.32 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 16.
Unsloth: vLLM's KV Cache can use up to 2.28 GB. Also sw

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-02 11:15:53 [gpu_model_runner.py:2392] Model loading took 3.3747 GiB and 3.275754 seconds
INFO 01-02 11:16:08 [backends.py:539] Using cache directory: /root/.cache/vllm/torch_compile_cache/392892021c/rank_0_0/backbone for vLLM's torch.compile
INFO 01-02 11:16:08 [backends.py:550] Dynamo bytecode transform time: 14.29 s


Unsloth: Compiling kernels: 100%|██████████| 7/7 [00:00<00:00, 500.45it/s, triton_poi_fused_view_6]

...

Some weights of Qwen3ForCausalLM were not initialized from the model checkpoint at unsloth/qwen3-4b-base-unsloth-bnb-4bit and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Performing substitution for additional_keys=set()
Unsloth: Just some info: will skip parsing ['layer_norm1', 'ffn_norm', 'post_feedforward_layernorm', 'cross_attn_input_layernorm', 'layer_norm2', 'q_norm', 'attention_norm', 'post_layernorm', 'norm1', 'norm', 'cross_attn_post_attention_layernorm', 'input_layernorm', 'pre_feedforward_layernorm', 'post_attention_layernorm', 'norm2', 'k_norm']
 Preparing 164 prompts...
 Running vLLM Batch Generation...


Adding requests:   0%|          | 0/164 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/164 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Saved 164 samples to: data/evaluation/generations/sft_cot.jsonl


In [29]:
# -------------------------
# RPO MODEL (Non-CoT)
# -------------------------
evaluate_model(
    model_path=GRPO_MODEL_PATH,
    problems=problems,
    task_ids=task_ids,
    use_cot=False,
    output_jsonl=str(GEN_DIR / "grpo_non_cot.jsonl"),
    max_new_tokens=MAX_NEW_TOKENS_NON_COT,
    temperature=TEMP_NON_COT,
)


 Loading: models/qwen3-4b-grpo-final | Mode: Non-CoT
INFO 01-02 10:46:53 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 01-02 10:46:53 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3. vLLM: 0.10.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-base-unsloth-bnb-4bit with actual GPU utilization = 20.41%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 79.32 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 64.
Unsloth: vLLM's KV Cache can use up to 13.3

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-02 10:47:16 [gpu_model_runner.py:2392] Model loading took 3.3988 GiB and 2.904477 seconds
INFO 01-02 10:47:31 [backends.py:539] Using cache directory: /root/.cache/vllm/torch_compile_cache/0550210604/rank_0_0/backbone for vLLM's torch.compile
INFO 01-02 10:47:31 [backends.py:550] Dynamo bytecode transform time: 14.14 s


Unsloth: Compiling kernels: 100%|██████████| 7/7 [00:00<00:00, 412.70it/s, triton_poi_fused_view_6]

...

Some weights of Qwen3ForCausalLM were not initialized from the model checkpoint at unsloth/qwen3-4b-base-unsloth-bnb-4bit and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Performing substitution for additional_keys=set()
Unsloth: Just some info: will skip parsing ['layer_norm1', 'ffn_norm', 'post_feedforward_layernorm', 'cross_attn_input_layernorm', 'layer_norm2', 'q_norm', 'attention_norm', 'post_layernorm', 'norm1', 'norm', 'cross_attn_post_attention_layernorm', 'input_layernorm', 'pre_feedforward_layernorm', 'post_attention_layernorm', 'norm2', 'k_norm']
 Preparing 164 prompts...
 Running vLLM Batch Generation...


Adding requests:   0%|          | 0/164 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/164 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Saved 164 samples to: data/evaluation/generations/grpo_non_cot.jsonl


In [26]:
# -------------------------
# GRPO MODEL (CoT)
# -------------------------
evaluate_model(
    model_path=GRPO_MODEL_PATH,
    problems=problems,
    task_ids=task_ids,
    use_cot=True,
    output_jsonl=str(GEN_DIR / "grpo_cot.jsonl"),
    max_new_tokens=2048, # Increased for safety, vLLM handles the speed
    temperature=TEMP_COT,
)


 Loading: models/qwen3-4b-grpo-final | Mode: CoT
INFO 01-02 10:12:47 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 01-02 10:12:47 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3. vLLM: 0.10.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-base-unsloth-bnb-4bit with actual GPU utilization = 69.6%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 79.32 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 52.34 GB

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 01-02 10:13:17 [__init__.py:1815] Using max model len 4096
WARNING 01-02 10:13:17 [_ipex_ops.py:16] Import error msg: No module named 'intel_extension_for_pytorch'
INFO 01-02 10:13:20 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 01-02 10:13:20 [lora.py:92] `lora_extra_vocab_size` is deprecated and will be removed in v0.12.0. Additional vocabulary support for LoRA adapters is being phased out.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.0.mlp', 'model.layers.4.mlp', 'model.layers.3.self_attn', 'model.layers.0.self_attn', 'model.layers.6.mlp', 'model.layers.1.self_attn', 

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Unsloth 2025.12.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


 Preparing 164 prompts...
 Running vLLM Batch Generation...


Adding requests:   0%|          | 0/164 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/164 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Saved 164 samples to: data/evaluation/generations/grpo_cot.jsonl


In [27]:
# REPLACE THIS with the path to the file you want to inspect
# Example: "generation_results/grpo_cot_2048.jsonl"
FILE_PATH = str(GEN_DIR / "grpo_cot.jsonl")

print(f" INSPECTING: {FILE_PATH}\n")

with open(FILE_PATH, 'r') as f:
    for line in f:
        data = json.loads(line)
        task_id = data.get("task_id", "Unknown Task")
        completion = data.get("completion", "")

        print(f" TASK: {task_id}")
        print("-" * 80)
        # Printing completion directly renders newlines correctly
        print(completion)
        print("=" * 80 + "\n")

 INSPECTING: data/evaluation/generations/grpo_cot.jsonl

 TASK: HumanEval/0
--------------------------------------------------------------------------------
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False


 TASK: HumanEval/1
--------------------------------------------------------------------------------
    paren_string = paren_string.replace(" ", "")
    groups = []
    counter = 0
    current_group = ""
    for char in paren_string:
        if char == "(":
            counter += 1
            current_group += char
        elif char == ")":
            counter -= 1
            current_group += char
            if counter == 0:
                groups.append(current_group)
                current_group = ""
    return groups


 TASK: HumanEval/2
--------------------------------------------------------------------------------
    integer_part = int(number)
